In [1]:
from typing import List

#### 1. [1409]. Queries on a Permutation With Key

In [81]:
class Solution:
    def processQueries(self, queries: List[int], m: int) -> List[int]:
        P = [*range(1,m+1)]
        res = []
        for val in queries:
            res.append(P.index(val))
            P.insert(0, P.pop(res[-1]))
        return res

In [82]:
class Fenwick:
    def __init__(self, n):
        sz = 1
        while sz <= n:
            sz *= 2
        self.size = sz
        self.data = [0] * sz

    def sum(self, i): # moving backward
        #print('sum: ', end=' ')
        s = 0
        while i > 0:
            s += self.data[i]
            i -= i & -i
        return s

    def add(self, i, x): # moving forward
        print('add: ', end=' ')
        while i < self.size:
            print(i, end=', ')
            self.data[i] += x
            i += i & -i
        print()

class Solution:    
    def processQueries(self, queries, n):
        fenw = Fenwick(2 * n)
        vimap = {}
        for i in range(1, n + 1):
            fenw.add(i + n, 1)
            vimap[i] = i + n
        print()
        print(fenw.data)
        print(vimap)
        cur = n
        ans = []
        for q in queries:
            i = vimap.pop(q)
            rank = fenw.sum(i-1)
            ans.append(rank)
            
            vimap[q] = cur
            fenw.add(i, -1)
            fenw.add(cur, 1)
            cur -= 1
            print(fenw.data)
        return ans, fenw

In [83]:
queries = [3,1,2,1]; m = 5
queries = [4,1,2,2]; m = 4
obj = Solution()
x = obj.processQueries(queries, m)

add:  5, 6, 8, 
add:  6, 8, 
add:  7, 8, 
add:  8, 

[0, 0, 0, 0, 0, 1, 2, 1, 4, 0, 0, 0, 0, 0, 0, 0]
{1: 5, 2: 6, 3: 7, 4: 8}
add:  8, 
add:  4, 8, 
[0, 0, 0, 0, 1, 1, 2, 1, 4, 0, 0, 0, 0, 0, 0, 0]
add:  5, 6, 8, 
add:  3, 4, 8, 
[0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0]
add:  6, 8, 
add:  2, 4, 8, 
[0, 0, 1, 1, 3, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0]
add:  2, 4, 8, 
add:  1, 2, 4, 8, 
[0, 1, 1, 1, 3, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0]


In [84]:
x[1].data

[0, 1, 1, 1, 3, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0]

In [85]:
x[0]

[3, 1, 2, 0]

#### 2. 

In [121]:
# Give TLE error as complexity would be n!/r!(n-r)!
from itertools import combinations
class Solution:
    def numTeams(self, rating: List[int]) -> int:
        count=0
        for idx, i in enumerate(iter(combinations(rating, 3))):
            if i[0]<i[1]<i[2] or i[0]>i[1]>i[2]:
                count+=1
        return count

In [133]:
# Another approach

class Solution:
    def numTeams(self, rating: List[int]) -> int:
        sequence = 0
        length = len(rating)
        for i in range(1, length-1):
            smaller_cnt, greater_cnt = 0,0
            for j in range(length):
                if j<i and rating[j]<rating[i]: # count all smallers
                    smaller_cnt+=1
                elif j>i and rating[j]>rating[i]: # count all greaters
                    greater_cnt+=1
            sequence+=smaller_cnt*greater_cnt
        
            smaller_cnt, greater_cnt = 0,0
            for j in range(length):
                if j<i and rating[j]>rating[i]: # count all greaters
                    smaller_cnt+=1
                elif j>i and rating[j]<rating[i]: # count all smallers
                    greater_cnt+=1
            sequence+=smaller_cnt*greater_cnt
        return sequence        

In [136]:
rating = [2,5,3,4,1]
rating = [2,1,3]
rating = [1,2,3,4]
Solution().numTeams(rating)

4

In [143]:
n = 2_00_000
n*(n+1)/2

20000100000.0

#### 3. Longest uploaded Prefix

In [222]:
class LUPrefix:

    def __init__(self, n: int):
        self.N = n
        self.bit = [0]*(self.N+1)
            
    def _sum(self, idx):
        ans = 0
        while idx>=1:
            ans+=self.bit[idx]
            idx-=(idx&-idx)
        return ans
        
    def upload(self, video: int) -> None:
        idx, val = video, video
        while idx<=self.N:
            self.bit[idx]+=val
            idx+=(idx&-idx)
    
    def longest(self) -> int:
        l,r = 1, self.N
        while l<=r:
            mid = (l+r)//2
            if self._sum(mid) == mid*(mid+1)//2:
                l = mid+1
            else:
                r = mid-1
        return l-1

In [224]:
class LUPrefix:

    def __init__(self, n: int):
        self._longest = 1
        self._nums = [False]*(n+2)

    def upload(self, video: int) -> None:
        self._nums[video] = True
        
        while self._nums[self._longest]:
            self._longest+=1
        
    def longest(self) -> int:
        return self._longest-1

In [225]:
n = 6
obj = LUPrefix(n)
obj.upload(3)
obj.longest()
obj.upload(1)
obj.longest()
obj.upload(2)
obj.longest()
#print()
#obj.bit
#param_2 = obj.longest()
#obj._sum(4)

3

#### 4. [1157]. Online Majority Element In Subarray

In [290]:
from collections import defaultdict
from bisect import bisect_left, bisect_right
from random import randint

class MajorityChecker:
    def __init__(self,arr):
        map =defaultdict(list)
        for i, x in enumerate(arr):
            map[x].append(i)
        self.arr, self.b = arr, map
        
    def query(self, left: int, right: int, threshold: int) -> int:
        for _ in range(20):
            a = self.arr[randint(left, right)]
            l =bisect_left(self.b[a], left)
            r =bisect_right(self.b[a], right)
            if r - l >= threshold: return a
        return -1

In [291]:
# Input
# ["MajorityChecker", "query", "query", "query"]
# [[[1, 1, 2, 2, 1, 1]], [0, 5, 4], [0, 3, 3], [2, 3, 2]]
# Output
# [null, 1, -1, 2]
arr = [1,1,2,2,1,1]
obj = MajorityChecker(arr)
obj.query(0,5,4)

1

#### 5. [2193]. Minimum Number of Moves to Make Palindrome

In [371]:
import math

class Solution(object):
    def minMovesToMakePalindrome(self, s: str) -> int:
        ans = 0
        n = len(s)
        target = n - 1 # 0's corresponding index
        for i in range(math.ceil(n / 2)):
            for j in range(target, i - 1, -1):
                # search backward
                if j == i:
                    # occurrance is 1, needs to be placed at center
                    ans += n // 2 - i
                elif s[i] == s[j]:
                    s = s[:j] + s[j + 1 : target + 1] + s[j] + s[target + 1:]
                    ans += target - j
                    target -= 1
                    break
        return ans

In [372]:
s = "aabb"
#s = "letelt"
Solution().minMovesToMakePalindrome(s)

2